In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

In [7]:
#df = pd.read_excel("IWA.xlsx", sheet_name='Final Raw Sample(3%)')
df = pd.read_excel("IWA.xlsx", sheet_name='Final Raw Sample(0%)')
df = df.dropna()
df["Revenue"] = df["Total Environmental Cost"]/df["Environmental Intensity (Sales)"]
df["Operating Income"] = df["Total Environmental Cost"]/df["Environmental Intensity (Op Inc)"]
df["Environmental Intensity (Op Inc)"] = df["Environmental Intensity (Op Inc)"]*100
df["Environmental Intensity (Sales)"] = df["Environmental Intensity (Sales)"]*100
df.head()

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed,Revenue,Operating Income
0,2019,SAGA PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Multi-line Insurance,Activities auxiliary to financial intermediati...,-2.887178,-13.025357,-3.184231e+07,-3.115075e+07,-7184.203318,...,-4.739468,-1.027193,-3584.970569,-5.649112,70.667599,70.667599,-1297.277948,0.006135,1.102887e+09,244464000.0
1,2019,BURSA MALAYSIA BHD,MALAYSIA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-1.677157,-3.465639,-1.968379e+06,-1.924910e+06,-451.342112,...,-1.410813,-1.207108,-222.196310,-1.681590,10.138780,10.138780,-79.398691,0.043215,1.173640e+08,56797000.0
2,2019,INTERTEK GROUP PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Research & Consulting Services,Activities auxiliary to financial intermediati...,-1.529690,-9.487849,-6.059927e+07,-5.928166e+07,-13774.014902,...,-17.024036,-3.689647,-6861.392776,-20.291452,253.836024,253.836024,-2470.054721,0.011467,3.961539e+09,638704000.0
4,2019,BUREAU VERITAS SA,FRANCE,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.699273,-5.095678,-3.997865e+07,-3.910761e+07,-9330.459280,...,-37.818819,-9.136488,-4606.916825,-45.077368,586.030400,586.030400,-1632.997165,0.033005,5.717172e+09,784560000.0
5,2019,RICARDO PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.665664,-7.274924,-3.247235e+06,-3.176408e+06,-753.133044,...,-2.096319,-0.454339,-373.412363,-2.498664,31.257053,31.257053,-132.536045,0.025666,4.878190e+08,44636000.0


In [8]:
df_GICSBrew = df[df['GICS Sub-Industry'] == 'Brewers']

# Group the data by "Company Name" and count unique years for each group
company_years_count = df_GICSBrew.groupby("Company Name")["Year"].nunique()

# Filter companies with at least 10 years of data
filtered_companies = company_years_count[company_years_count >= 10].index

# Create a new DataFrame with only the companies that meet the criteria
df_GICSBrewFiltered = df_GICSBrew[df_GICSBrew["Company Name"].isin(filtered_companies)]

df_GICSBrewFiltered.head()

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed,Revenue,Operating Income
517,2019,MOLSON COORS BEVERAGE CO,UNITED STATES OF AMERICA,Brewers,Manufacture of beverages,-6.596233,-46.522794,-6.978419e+08,-2.950986e+08,-77656.206578,...,-1113.208360,-326.988191,-37024.568588,-1326.865949,15176.840942,15176.840942,-12261.768244,0.059202,1.057940e+10,1.500000e+09
523,2019,ANHEUSER-BUSCH INBEV,BELGIUM,Brewers,Manufacture of beverages,-3.388736,-11.492977,-1.778566e+09,-1.561186e+09,-391750.191978,...,-3914.291085,-325.789140,-190769.424529,-4665.559245,56659.806086,56659.806086,-64862.799133,0.080199,5.248464e+10,1.547524e+10
526,2019,CARLSBERG A/S,DENMARK,Brewers,Manufacture of beverages,-2.414563,-16.054846,-2.388141e+08,-1.962082e+08,-60262.712495,...,-1480.449400,-288.844325,-27951.868900,-1764.591400,19253.589023,19132.282417,-8148.334758,0.029599,9.890572e+09,1.487489e+09
530,2019,HEINEKEN HOLDING NV,NETHERLANDS,Brewers,Manufacture of beverages,-1.898778,-12.419412,-5.102237e+08,-4.329371e+08,-113003.339951,...,-1458.522182,-168.769751,-54528.158099,-1738.455701,20656.408130,20656.408130,-18050.980779,0.093336,2.687117e+10,4.108276e+09
532,2019,HEINEKEN NV,NETHERLANDS,Brewers,Manufacture of beverages,-1.709847,-10.508374,-4.594560e+08,-4.332461e+08,-113074.003951,...,-1458.522182,-168.769751,-54563.490099,-1738.455701,20656.408130,20656.408130,-18063.841679,0.101854,2.687117e+10,4.372284e+09


In [11]:
harvard_set = ["Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]

sdg_set = ["SDG 1.5", "SDG 2.1", "SDG 2.2", "SDG 2.3", "SDG 2.4", "SDG 3.3", 
                         "SDG 3.4", "SDG 3.9", "SDG 6", "SDG 12.2", "SDG 14.1", "SDG 14.2", 
                         "SDG 14.3", "SDG 14.c", "SDG 15.1", "SDG 15.2", "SDG 15.5"]

y_sales = df_GICSBrewFiltered["Revenue"]
y_opinc = df_GICSBrewFiltered["Operating Income"]

X_harvard = df_GICSBrewFiltered[harvard_set]
X_harvard = sm.add_constant(X_harvard)

X_sdg = df_GICSBrewFiltered[sdg_set]
X_sdg = sm.add_constant(X_sdg)

In [18]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_harvard, y_opinc, test_size=0.2, random_state=42)

# Create and fit the Lasso regression model
alpha = 1.0  # You can adjust the regularization strength (alpha) as needed
lasso_model = Lasso(alpha=alpha)
lasso_model.fit(X_train, y_train)

# Get the coefficients (weights) of the independent variables
coefficients = lasso_model.coef_

# Print the coefficients
print("Lasso Coefficients:")
for i in range(len(coefficients)):
    if i == 0: continue
    print(f"{harvard_set[i-1]}: {coefficients[i]:.4f}")

Lasso Coefficients:
Working Capacity: -8.3573
Fish Production Capacity: 89296.2392
Crop Production Capacity: -1981.6318
Meat Production Capacity: 1805.8892
Biodiversity: 84333.4969
Abiotic Resources: -246.1929
Water production capacity (Drinking water & Irrigation Water): -0.6835
Wood Production Capacity: 70548.0634


c:\Users\ianpe\mambaforge\envs\PythonBase\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.243e+19, tolerance: 8.290e+16
  model = cd_fast.enet_coordinate_descent(


In [19]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_sdg, y_opinc, test_size=0.2, random_state=42)

# Create and fit the Lasso regression model
alpha = 1.0  # You can adjust the regularization strength (alpha) as needed
lasso_model = Lasso(alpha=alpha)
lasso_model.fit(X_train, y_train)

# Get the coefficients (weights) of the independent variables
coefficients = lasso_model.coef_

# Print the coefficients
print("Lasso Coefficients:")
for i in range(len(coefficients)):
    if i == 0: continue
    print(f"{sdg_set[i-1]}: {coefficients[i]:.4f}")

Lasso Coefficients:
SDG 1.5: -14.4748
SDG 2.1: -8.0638
SDG 2.2: 0.5085
SDG 2.3: 48.2679
SDG 2.4: -2.4765
SDG 3.3: -9.0308
SDG 3.4: -199.0191
SDG 3.9: 188858.8116
SDG 6: 3.8241
SDG 12.2: -15226.4886
SDG 14.1: -4980411.8815
SDG 14.2: 427323.3775
SDG 14.3: 8722.7660
SDG 14.c: -339608.8375
SDG 15.1: -49520.8822
SDG 15.2: -3508.9641
SDG 15.5: 108110.7075


c:\Users\ianpe\mambaforge\envs\PythonBase\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.607e+18, tolerance: 8.290e+16
  model = cd_fast.enet_coordinate_descent(


In [20]:
# Fit the random-effects panel regression model
model = sm.OLS(y_opinc, X_harvard).fit()

# Get the regression results
results = model.summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:       Operating Income   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     965.0
Date:                Thu, 05 Oct 2023   Prob (F-statistic):           3.34e-61
Time:                        12:43:57   Log-Likelihood:                -1469.7
No. Observations:                  70   AIC:                             2957.
Df Residuals:                      61   BIC:                             2978.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                                                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------

In [21]:
# Fit the random-effects panel regression model
model = sm.OLS(y_opinc, X_sdg).fit()

# Get the regression results
results = model.summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:       Operating Income   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                     970.0
Date:                Thu, 05 Oct 2023   Prob (F-statistic):           1.52e-61
Time:                        12:44:13   Log-Likelihood:                -1460.6
No. Observations:                  70   AIC:                             2943.
Df Residuals:                      59   BIC:                             2968.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -3.72e+07   1.74e+08     -0.213      0.8

## FIXED EFFECTS

In [124]:
import pandas as pd
from linearmodels.panel import PanelOLS



Company Name  \
Industry (Exiobase)                                Year                             
Activities auxiliary to financial intermediatio... 2019-01-01            SAGA PLC   
                                                   2019-01-01  BURSA MALAYSIA BHD   
                                                   2019-01-01  INTERTEK GROUP PLC   
                                                   2019-01-01         JSE LIMITED   
                                                   2019-01-01   BUREAU VERITAS SA   

                                                                                                         Country  \
Industry (Exiobase)                                Year                                                            
Activities auxiliary to financial intermediatio... 2019-01-01  UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...   
                                                   2019-01-01                                           MALAYSIA   
                                                   2019-01-01  UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...   
                                                   2019-01-01                                       SOUTH AFRICA   
                                                   2019-01-01                                             FRANCE   

                                                                            GICS Sub-Industry  \
Industry (Exiobase)                                Year                                         
Activities auxiliary to financial intermediatio... 2019-01-01            Multi-line Insurance   
                                                   2019-01-01      Financial Exchanges & Data   
                                                   2019-01-01  Research & Consulting Services   
                                                   2019-01-01      Financial Exchanges & Data   
                                                   2019-01-01  Research & Consulting Services   

                                                               Environmental Intensity (Sales)  \
Industry (Exiobase)                                Year                                          
Activities auxiliary to financial intermediatio... 2019-01-01                        -2.887178   
                                                   2019-01-01                        -1.677157   
                                                   2019-01-01                        -1.529690   
                                                   2019-01-01                        -1.462497   
                                                   2019-01-01                        -0.699273   

                                                               Environmental Intensity (Op Inc)  \
Industry (Exiobase)                                Year                                           
Activities auxiliary to financial intermediatio... 2019-01-01                        -13.025357   
                                                   2019-01-01                         -3.465639   
                                                   2019-01-01                         -9.487849   
                                                   2019-01-01                               NaN   
                                                   2019-01-01                         -5.095678   

                                                               Total Environmental Cost  \
Industry (Exiobase)                                Year                                   
Activities auxiliary to financial intermediatio... 2019-01-01             -3.184231e+07   
                                                   2019-01-01             -1.968379e+06   
                                                   2019-01-01             -6.059927e+07   
                                                   2019-01-01             -2.290124e+06   
                                                   20

In [129]:
fixedEffects_df = df.copy()
fixedEffects_df['Year'] = pd.to_datetime(fixedEffects_df['Year'], format='%Y')

# Set 'Year' as the time variable and 'Company Name' as the entity variable
fixedEffects_df.set_index(['Industry (Exiobase)', 'Year'], inplace=True)

y = fixedEffects_df['Environmental Intensity (Sales)']
X = fixedEffects_df[["Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]]

fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True, check_rank=False)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary

Dep. Variable:,Environmental Intensity (Sales),R-squared:,0.1826
Estimator:,PanelOLS,R-squared (Between):,0.2766
No. Observations:,14035,R-squared (Within):,0.1823
Date:,"Mon, Oct 02 2023",R-squared (Overall):,0.2603
Time:,16:00:24,Log-likelihood,-6.088e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,389.99
Entities:,56,P-value,0.0000
Avg Obs:,250.62,Distribution:,"F(8,13962)"
Min Obs:,1.0000,,
Max Obs:,1463.0,F-statistic (robust):,389.99


In [130]:
fixedEffects_df = df.copy()
fixedEffects_df['Year'] = pd.to_datetime(fixedEffects_df['Year'], format='%Y')

# Set 'Year' as the time variable and 'Company Name' as the entity variable
fixedEffects_df.set_index(['GICS Sub-Industry', 'Year'], inplace=True)
#fixedEffects_df["Revenue"] = fixedEffects_df["Total Environmental Cost"]/fixedEffects_df["Environmental Intensity (Sales)"]
fixedEffects_df.head()

# Define the dependent variable (Y) and independent variables (X1-X15)
#y = np.log(fixedEffects_df['Revenue'])
y = fixedEffects_df['Environmental Intensity (Sales)']
X = fixedEffects_df[["Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]]

fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True, check_rank=False)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary

Dep. Variable:,Environmental Intensity (Sales),R-squared:,0.1733
Estimator:,PanelOLS,R-squared (Between):,0.2989
No. Observations:,14035,R-squared (Within):,0.1730
Date:,"Mon, Oct 02 2023",R-squared (Overall):,0.2587
Time:,16:00:31,Log-likelihood,-6.03e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,363.28
Entities:,153,P-value,0.0000
Avg Obs:,91.732,Distribution:,"F(8,13865)"
Min Obs:,1.0000,,
Max Obs:,521.00,F-statistic (robust):,363.28


In [143]:
X = df[["Year"]]
y = np.log(df[["Environmental Intensity (Sales)"]])
X = sm.add_constant(X)

# Fit the random-effects panel regression model
model = sm.OLS(y, X).fit()

# Get the regression results
results = model.summary()
print(results)

                                   OLS Regression Results                                  
Dep. Variable:     Environmental Intensity (Sales)   R-squared:                         nan
Model:                                         OLS   Adj. R-squared:                    nan
Method:                              Least Squares   F-statistic:                       nan
Date:                             Mon, 02 Oct 2023   Prob (F-statistic):                nan
Time:                                     16:17:48   Log-Likelihood:                    nan
No. Observations:                            14035   AIC:                               nan
Df Residuals:                                14033   BIC:                               nan
Df Model:                                        1                                         
Covariance Type:                         nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-

c:\Users\ianpe\mambaforge\envs\PythonBase\Lib\site-packages\pandas\core\internals\blocks.py:366: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
